# Import Libraries

In [ ]:
# Install the 'resampy' library, which is used for audio resampling
!pip install resampy

In [ ]:
# Installing 
import numpy as np 
import pandas as pd
import os
import librosa
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import IPython
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Load Dataset

In [ ]:
# Define the path to the dataset
audio_files_path = "Path to your dataset" 

In [ ]:
# Load and preprocess audio files using librosa
folders = os.listdir(audio_files_path)
 print(folders)

In [ ]:
#paths to real and fake audio file
real_audio = "Path to real audio from dataset"
fake_audio = "Path to fake audio from dataset"

# Visualization of Dataset 

In [ ]:
#Print real Audio file
print("Real Audio:")
IPython.display.Audio(real_audio) 

In [ ]:
#Print fake Audio file
print("Fake Audio:")
IPython.display.Audio(fake_audio)

In [ ]:
# Load and visualize the waveform of a real audio sample
real_ad, real_sr = librosa.load(real_audio)
plt.figure(figsize=(12, 4))
plt.plot(real_ad)
plt.title("Real Audio Data")
plt.show()

In [ ]:
# Generate and display the spectrogram of a real audio sample
real_spec = np.abs(librosa.stft(real_ad))
real_spec = librosa.amplitude_to_db(real_spec, ref=np.max)
plt.figure(figsize=(14, 5))
librosa.display.specshow(real_spec, sr=real_sr, x_axis="time", y_axis="log")
plt.colorbar(format="%+2.0f dB")
plt.title("Real Audio Spectogram")
 plt.show()

In [ ]:
# Generate and display the Mel spectrogram of a real audio sample
real_mel_spect = librosa.feature.melspectrogram(y=real_ad, sr=real_sr)
real_mel_spect = librosa.power_to_db(real_mel_spect, ref=np.max)
plt.figure(figsize=(14, 5))
librosa.display.specshow(real_mel_spect, y_axis="mel", x_axis="time")
plt.title("Real Audio Mel Spectogram")
plt.colorbar(format="%+2.0f dB")
plt.show()

In [ ]:
# Generate and display the chromagram of a real audio sample
real_chroma = librosa.feature.chroma_cqt(y=real_ad, sr=real_sr, bins_per_octave=36)
plt.figure(figsize=(14, 5))
librosa.display.specshow(real_chroma, sr=real_sr, x_axis="time", y_axis="chroma", vmin=0, vmax=1)
plt.colorbar()
plt.title("Real Audio Chromagram")
plt.show()


In [ ]:
# Extract MFCC features from the audio and display them as a time-series spectrogram.
real_mfccs = librosa.feature.mfcc(y=real_ad, sr=real_sr)
plt.figure(figsize=(14, 5))
librosa.display.specshow(real_mfccs, sr=real_sr, x_axis="time")
plt.colorbar()
plt.title("Real Audio Mel-Frequency Cepstral Coefficients (MFCCs)")
plt.show()

In [ ]:
# Load and plot the fake audio data to visualize the waveform.
fake_ad, fake_sr = librosa.load(fake_audio)
plt.figure(figsize=(12, 4))
plt.plot(fake_ad)
plt.title("Fake Audio Data")
plt.show()

In [ ]:
# Compute and display the spectrogram of the fake audio using the Short-Time Fourier Transform (STFT).
fake_spec = np.abs(librosa.stft(fake_ad))
fake_spec = librosa.amplitude_to_db(fake_spec, ref=np.max)
plt.figure(figsize=(14, 5))
librosa.display.specshow(fake_spec, sr=fake_sr, x_axis="time", y_axis="log")
plt.colorbar(format="%+2.0f dB")
plt.title("Fake Audio Spectogram")
plt.show()

In [ ]:
# Compute and display the spectrogram of the fake audio using the Short-Time Fourier Transform (STFT).
fake_mel_spect = librosa.feature.melspectrogram(y=fake_ad, sr=fake_sr)
fake_mel_spect = librosa.power_to_db(fake_mel_spect, ref=np.max)
plt.figure(figsize=(14, 5))
librosa.display.specshow(fake_mel_spect, y_axis="mel", x_axis="time")
plt.title("Fake Audio Mel Spectogram")
plt.colorbar(format="%+2.0f dB")
plt.show()

In [ ]:
# Extract and visualize the chromagram of the fake audio to show pitch class distribution over time.
fake_chroma = librosa.feature.chroma_cqt(y=fake_ad, sr=fake_sr, bins_per_octave=36)
plt.figure(figsize=(14, 5))
librosa.display.specshow(fake_chroma, sr=fake_sr, x_axis="time", y_axis="chroma", vmin=0, vmax=1)
plt.colorbar()
plt.title("Fake Audio Chromagram")
plt.show()

In [ ]:
# Extract and display the MFCCs of the fake audio to analyze its timbral features over time.
fake_mfccs = librosa.feature.mfcc(y=fake_ad, sr=fake_sr)
plt.figure(figsize=(14, 5))
librosa.display.specshow(fake_mfccs, sr=fake_sr, x_axis="time")
plt.colorbar()
plt.title("Fake Audio Mel-Frequency Cepstral Coefficients (MFCCs)")
plt.show()

# Preprocessing of the dataset

In [ ]:
# Load audio files from each folder, extract MFCC features, and store them with their corresponding labels.  
data = []
labels = []
for folder in folders:
files = os.listdir(os.path.join(audio_files_path, folder))
for file in tqdm(files):
file_path = os.path.join(audio_files_path, folder, file)
audio, sample_rate = librosa.load(file_path, res_type="kaiser_fast")
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_features_scaled = np.mean(mfccs_features.T, axis=0)
data.append(mfccs_features_scaled)
labels.append(folder)

In [ ]:
# Create a DataFrame to organize the extracted features and their corresponding class labels.  
feature_df = pd.DataFrame({"features": data, "class": labels})
feature_df.head()

In [ ]:
# Display the count of samples for each class label in the dataset.  
feature_df["class"].value_counts()

In [ ]:
# Encode categorical class labels into numerical format using LabelEncoder.  
def label_encoder(column):
    le = LabelEncoder().fit(column)
    print(column.name, le.classes_)
    return le.transform(column)

In [ ]:
# Apply label encoding to convert class labels in the DataFrame to numeric values.  
feature_df["class"] = label_encoder(feature_df["class"])

# Feature Scaling

In [ ]:
# Convert the features and class labels from the DataFrame into NumPy arrays for model training.  
X = np.array(feature_df["features"].tolist())
y = np.array(feature_df["class"].tolist())

In [ ]:
# Apply random oversampling to balance the class distribution in the dataset.  
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

In [ ]:
# Convert the resampled class labels to one-hot encoded format for model training.  
y_resampled = to_categorical(y_resampled)

In [ ]:
# Split the resampled dataset into training and testing sets (80% train, 20% test).  
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [ ]:
# Determine the number of unique class labels in the dataset.  
num_labels = len(feature_df["class"].unique())
num_labels

In [ ]:
# Retrieve the shape of a single feature vector to define the input shape for the model.  
input_shape = feature_df["features"][0].shape
input_shape

# Building the Model

In [ ]:
# Define a neural network model with multiple layers, including dense, activation, and dropout layers for regularization.
model = Sequential()
model.add(Dense(128, input_shape=input_shape))
model.add(Activation(activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Activation(activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(128))
model.add(Activation(activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(num_labels))
model.add(Activation(activation="softmax"))

In [ ]:
# Compile the model using categorical crossentropy loss, Adam optimizer, and accuracy as the evaluation metric.\
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Display the summary of the model architecture, including layer details and parameter counts.
model.summary()

In [ ]:
# Train the model on the training data with validation, specifying batch size and number of epochs.
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=2, epochs=100) # callbacks=[early])


In [ ]:
# Evaluate the trained model on the test set and print the test loss and accuracy.
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

In [ ]:
# Plot the training and validation accuracy over epochs to visualize model performance.
plt.figure()
plt.title("Model Accuracy")
plt.plot(history.history["accuracy"], label="train")
plt.plot(history.history["val_accuracy"], label="validation")
plt.legend()
plt.ylim([0, 1])
plt.show()

In [ ]:
# Plot the training and validation loss over epochs to visualize model convergence.
plt.title("Model Loss")
plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="validation")
plt.legend()
plt.ylim([0, 1])
plt.show()

# Testing the Model

In [ ]:
# Define a function to detect whether an audio file is fake or real using the trained model and MFCC features.
def detect_fake(filename):
sound_signal, sample_rate = librosa.load(filename, res_type="kaiser_fast")
mfcc_features = librosa.feature.mfcc(y=sound_signal, sr=sample_rate, n_mfcc=40)
mfccs_features_scaled = np.mean(mfcc_features.T, axis=0)
mfccs_features_scaled = mfccs_features_scaled.reshape(1, -1)
result_array = model.predict(mfccs_features_scaled)
print(result_array)
result_classes = ["FAKE", "REAL"]
result = np.argmax(result_array[0])
print("Result:", result_classes[result])

In [ ]:
# Define file paths for the real and fake audio samples to be tested.
test_real = "Path to Original audio from Dataset"
test_fake = "Path to Original audio from Dataset" 

In [ ]:
# Call the detect_fake function to predict whether the real audio sample is fake or real.
detect_fake(test_real)

In [ ]:
# Call the detect_fake function to predict whether the fake audio sample is fake or real.
detect_fake(test_fake)